# Index
* [Import](#index)
* [Tokenization](#Tokenization)
* [CountVectorizer](#CountVectorizer)
* [PreprocessPipeline](#PreprocessPipeline)
* [OvsRest](#OvsRest)
    - [LogisticRegression](#LogisticRegression)
    - [SVC](#SVC)
* [Word2Vec](#Word2Vec)
* [BERT](#BERT)
* [USE](#USE)

In [1]:
import time

# from joblib import dump, load, Memory
import joblib

# import warnings

import numpy as np
import pandas as pd
import gensim
import matplotlib.pyplot as plt

# from gensim import (
#     corpora,
# )  # https://pypi.org/project/gensim/ #https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#sphx-glr-auto-examples-core-run-topics-and-transformations-py
from sklearn import (
    metrics,
    feature_extraction,
    linear_model,
    model_selection,
    preprocessing,
    cluster,
    decomposition,
    multiclass,
    svm,
    pipeline,
    exceptions,
    manifold,
)
from sklearn.utils._testing import ignore_warnings
from tensorflow import keras


# import nlpk

In [2]:
class colors:
    PURPLE = "\033[95m"
    BLUE = "\033[94m"
    CYAN = "\033[96m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

test https://scikit-learn.org/stable/modules/multiclass.html autre stategie que OneVsRest
https://keras.io/examples/nlp/text_extraction_with_bert/

In [3]:
pip install -q jupyter-black jupyter

Note: you may need to restart the kernel to use updated packages.


In [4]:
# for black formating
import jupyter_black  # pip install jupyter-black jupyter

jupyter_black.load()
# ??jupyter_black.load #show config

In [5]:
LONG = False
df = pd.read_csv(
    "/kaggle/input/p5-nlp/stackoverflowCleaned.csv", index_col=0, low_memory=False
)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118442 entries, 0 to 118441
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Id           118442 non-null  int64 
 1   Title        118442 non-null  object
 2   Body         118442 non-null  object
 3   0            118440 non-null  object
 4   1            118383 non-null  object
 5   2            118292 non-null  object
 6   3            118292 non-null  object
 7   4            76389 non-null   object
 8   5            4 non-null       object
 9   Title_clean  118389 non-null  object
 10  Code         88202 non-null   object
 11  Body_clean   118392 non-null  object
dtypes: int64(1), object(11)
memory usage: 11.7+ MB


# Tokenization

In [6]:
from ipywidgets import interact


@interact(i=(0, df.shape[0] - 1))
def test_tr_print(i):
    print(
        f'Title:{df["Title"][i]}\n{"*"*30}\nTitle_clean:{df["Title_clean"][i]}\n{"*"*30}\nBody:{df["Body"][i]}\n{"*"*30}\nBody_clean:{df["Body_clean"][i]}\n{"*"*30}\nCode:{df["Code"][i]}\n{"*"*30}\nTags:{df.loc[i,"0"]} {df.loc[i,"1"]} {df.loc[i,"2"]} {df.loc[i,"3"]} {df.loc[i,"4"]}'
    )

interactive(children=(IntSlider(value=59220, description='i', max=118441), Output()), _dom_classes=('widget-in…

In [7]:
sum(df["Title_clean"].isna())

53

In [8]:
df[df["Title_clean"].isna()][
    ["Title", "Title_clean", "Body", "Body_clean", "0", "1", "2", "3", "4"]
].head(5)

,Title,Title_clean,Body,Body_clean,0,1,2,3,4
302,Is < faster than <=?,NaN,<p>Is <code>if (a &lt; 901)</code> faster than...,example performance change loop code suppose g...,c++,c,performance,assembly,relational-operators
393,What is setup.py?,NaN,<p>What is <code>setup.py</code> and how can i...,configure,python,pypi,setup.py,python-packaging,NaN
1437,What is related_name used for?,NaN,<p>What is the <code>related_name</code> argum...,argument manytomanyfield foreignkey field exam...,python,django,foreign-keys,many-to-many,NaN
4062,"What is ""android:allowBackup""?",NaN,"<p>Since the <a href=""http://tools.android.com...",adt preview version version lint warning tell ...,android,adt,compiler-warnings,android-lint,android-backup-service
4870,Why is 0 < -0x80000000?,NaN,<p>I have below a simple program:</p>\n\n<pre>...,program condition if(bal int32_min work change...,c,signed,numeric-limits,numeric-conversion,NaN


In [9]:
print(sum(df["Body_clean"].isna()))
df[df["Body_clean"].isna()][
    ["Title", "Title_clean", "Body", "Body_clean", "Code"]
].head(5)

50


,Title,Title_clean,Body,Body_clean,Code
5,What and where are the stack and heap?,stack heap,<ul>\n<li>What are the stack and heap?</li>\n<...,NaN,NaN
168,What is The Rule of Three?,rule,<ul>\n<li>What does <em>copying an object</em>...,NaN,NaN
828,Why are these constructs using pre and post-in...,construct pre behavior,<pre><code>#include &lt;stdio.h&gt;\n\nint mai...,NaN,#include <stdio.h>\n\nint main(void)\n{\n in...
1207,How to use ADB Shell when Multiple Devices are...,use adb shell multiple device connect fail err...,<pre><code>$ adb --help\n</code></pre>\n<hr />...,NaN,$ adb --help\n -s SERIAL use device with give...
1640,What are the advantages of list initialization...,advantage list initialization brace,<pre><code>MyClass a1 {a}; // clearer and ...,NaN,MyClass a1 {a}; // clearer and less error-...


In [10]:
len(df[df["Body_clean"].isna() & df["Title_clean"].isna()])

0

In [11]:
sum((df["Title_clean"] + df["Body_clean"]).isna())

103

In [12]:
def tag_is_in(df: pd.DataFrame(), tag: str, nb_cols: int = 6):
    return eval("|".join(f'(df["{i}"] == "{tag}")' for i in range(0, nb_cols)))


def tags_are_in(df: pd.DataFrame(), tags: list):
    return pd.DataFrame({tag: tag_is_in(df, tag) for tag in tags})


# tags_are_in(df, ["javascript","java"])
tags_are_in(
    df[10001:15050], ["javascript", "java", "c#", "python", "html", "git", "php"]
)

,javascript,java,c#,python,html,git,php
10001,True,False,False,False,True,False,False
10002,False,True,False,False,False,False,False
10003,False,False,True,False,False,False,False
10004,False,False,False,False,True,False,False
10005,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
15045,False,False,False,False,False,False,False
15046,False,False,False,False,False,False,False
15047,True,False,False,False,False,False,False
15048,False,False,False,False,False,False,False


In [13]:
tags = df[["0", "1", "2", "3", "4", "5"]].stack()
target_names = [
    tag
    for tag, freq in tags.reset_index(drop=True).value_counts(normalize=True).items()
    if freq > 0.002
]
if not LONG:
    target_names = target_names[0:20] + ["git"]
target_names

['java',
 'c#',
 'javascript',
 'python',
 'android',
 'c++',
 'ios',
 'html',
 'php',
 '.net',
 'jquery',
 'css',
 'objective-c',
 'c',
 'sql',
 'iphone',
 'asp.net',
 'mysql',
 'linux',
 'node.js',
 'git']

# CountVectorizer

In [14]:
df["Token"] = df["Title_clean"].fillna("") + df["Body_clean"].fillna("")
print(
    f"{len(df[[len(token.split()) > 1 for token in df['Token']]])/len(df):.5%} des questions ont plus d'un tokens"
    if len(df[df["Token"] == ""]) == 0
    else f"{len(df[df['Token'] == ''])} questions n'ont pas de token"
)
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    df["Token"], tags_are_in(df, target_names), random_state=42
)

99.99409% des questions ont plus d'un tokens


In [15]:
for column in y_train.columns:
    t_train = y_train[column].value_counts(normalize=True)
    t_test = y_test[column].value_counts(normalize=True)
    print(f"{colors.BOLD}{column}{colors.ENDC}")
    print(
        f"\t Sur entrainement {colors.GREEN}{t_train[True]:.2%}{colors.ENDC} positif {colors.RED}{t_train[False]:.2%}{colors.ENDC} negatif"
    )
    print(
        f"\t Sur test {colors.GREEN}{t_test[True]:.2%}{colors.ENDC} positif {colors.RED}{t_test[False]:.2%}{colors.ENDC} negatif"
    )
    print("\n")

java
	 Sur entrainement 12.05% positif 87.95% negatif
	 Sur test 12.02% positif 87.98% negatif


c#
	 Sur entrainement 11.80% positif 88.20% negatif
	 Sur test 11.73% positif 88.27% negatif


javascript
	 Sur entrainement 11.77% positif 88.23% negatif
	 Sur test 11.33% positif 88.67% negatif


python
	 Sur entrainement 8.61% positif 91.39% negatif
	 Sur test 8.82% positif 91.18% negatif


android
	 Sur entrainement 7.00% positif 93.00% negatif
	 Sur test 6.91% positif 93.09% negatif


c++
	 Sur entrainement 6.63% positif 93.37% negatif
	 Sur test 6.63% positif 93.37% negatif


ios
	 Sur entrainement 6.25% positif 93.75% negatif
	 Sur test 6.21% positif 93.79% negatif


html
	 Sur entrainement 5.85% positif 94.15% negatif
	 Sur test 5.93% positif 94.07% negatif


php
	 Sur entrainement 5.56% positif 94.44% negatif
	 Sur test 5.49% positif 94.51% negatif


.net
	 Sur entrainement 5.34% positif 94.66% negatif
	 Sur test 5.30% positif 94.70% negatif


jquery
	 Sur entrainement 5.36% positi

In [16]:
best_min_df = 0.00001
best_max_df = 0.995
for min_df in [0.01, 0.05, 0.005, 0.0005, 0.00001] if LONG else [best_min_df]:
    for max_df in [0.95, 0.995, 0.9995] if LONG else [best_max_df]:
        print(f"\n{'*'*30}\n pour min_df:{min_df}; max_df:{max_df}")
        count_vectorizer = feature_extraction.text.CountVectorizer(
            min_df=min_df, max_df=max_df, max_features=None
        )
        #         pipe = pipeline.make_pipeline(feature_extraction.text.CountVectorizer(min_df=min_df, max_df=max_df, max_features=None), feature_extraction.text.TfidfTransformer( norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=False))
        #         feat_train = pipe.transform(X_train["Token"])
        #         print(feat_train.shape)

        token_vectors = count_vectorizer.fit_transform(df["Token"])
        print(
            f"\ttokens unique:\n\t\t{token_vectors.shape[1]} dans title et body regroupé"
        )


******************************
 pour min_df:1e-05; max_df:0.995
	tokens unique:
		53999 dans title et body regroupé


In [17]:
%%time
if LONG:
    truncSvd = decomposition.TruncatedSVD(n_components=1000).fit(
        preprocessing.StandardScaler(with_mean=False).fit_transform(token_vectors)
    )
    plt.figure(figsize=(30, 15))
    scree = truncSvd.explained_variance_ratio_ * 100
    plt.bar(np.arange(len(scree)) + 1, scree)
    plt.plot(np.arange(len(scree)) + 1, scree.cumsum(), c="red", marker="o")
    plt.xlabel("rang de l'axe d'inertie")
    plt.ylabel("pourcentage variance expliquée")
    plt.title("Eboulis des valeurs propres")
    plt.show()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


# PreprocessPipeline

In [18]:
pipeline_tfidf = pipeline.Pipeline(
    [
        (
            "count",
            feature_extraction.text.CountVectorizer(
                min_df=best_min_df, max_df=best_max_df, max_features=None
            ),
        ),
        (
            "tfidf",
            feature_extraction.text.TfidfTransformer(
                norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=False
            ),
        ),
        ("scale", preprocessing.StandardScaler(with_mean=False)),
        (
            "dimReduction",
            decomposition.TruncatedSVD(n_components=1_000),
        ),  # 1_000 = 10% de la variance
    ]
)
## @TODO cache
# memory = joblib.Memory(location="/kaggle/working/cache_dir", verbose=0)
# pipeline_tfidf = memory.cache(pipeline_tfidf)

# OvsRest

In [19]:
def save_score(y_true, y_pred, target_names, name) -> pd.DataFrame():
    scores = pd.DataFrame(
        metrics.classification_report(
            y_true, y_pred, target_names=target_names, zero_division=0, output_dict=True
        )
    ).T
    scores.to_csv(f"/kaggle/working/{name}_score.csv")
    return scores

## LogisticRegression

In [20]:
%%time
# with ignore_warnings(category=exceptions.ConvergenceWarning):
#     clf_lr = multiclass.OneVsRestClassifier(
#         linear_model.LogisticRegression(random_state=42)
#     ).fit(X_train, y_train)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [21]:
pipeline_logisticReg = pipeline.Pipeline(
    [
        ("tfidf", pipeline_tfidf),
        (
            "OvRLogisticReg",
            multiclass.OneVsRestClassifier(
                linear_model.LogisticRegression(random_state=42)
            ),
        ),
    ]
)

In [22]:
%%time
with ignore_warnings(category=exceptions.ConvergenceWarning):
    pipeline_logisticReg.fit(X_train, y_train)

CPU times: user 10min 31s, sys: 1min 26s, total: 11min 58s
Wall time: 3min 34s


In [23]:
pred_lr = pipeline_logisticReg.predict(X_test)

In [24]:
scores_lr = save_score(y_test, pred_lr, target_names, "LogisticRegression")
joblib.dump(pipeline_logisticReg, "LogisticRegression_model.joblib")
scores_lr.sort_values(by="support", ascending=False)

,precision,recall,f1-score,support
samples avg,0.062856,0.058837,0.058605,33044.0
micro avg,0.471857,0.062916,0.111028,33044.0
weighted avg,0.379913,0.062916,0.105741,33044.0
macro avg,0.289857,0.044122,0.074589,33044.0
java,0.591281,0.121979,0.202237,3558.0
c#,0.496109,0.073424,0.127916,3473.0
javascript,0.381250,0.036353,0.066376,3356.0
python,0.592841,0.101455,0.173259,2612.0
android,0.721781,0.222005,0.339566,2045.0
c++,0.438438,0.074338,0.127122,1964.0


In [25]:
scores_lr.sort_values(by="precision", ascending=False)

,precision,recall,f1-score,support
android,0.721781,0.222005,0.339566,2045.0
ios,0.595930,0.111413,0.187729,1840.0
python,0.592841,0.101455,0.173259,2612.0
java,0.591281,0.121979,0.202237,3558.0
c#,0.496109,0.073424,0.127916,3473.0
micro avg,0.471857,0.062916,0.111028,33044.0
c++,0.438438,0.074338,0.127122,1964.0
javascript,0.381250,0.036353,0.066376,3356.0
weighted avg,0.379913,0.062916,0.105741,33044.0
php,0.324503,0.030117,0.055118,1627.0


In [26]:
scores_lr.sort_values(by="recall", ascending=False)

,precision,recall,f1-score,support
android,0.721781,0.222005,0.339566,2045.0
java,0.591281,0.121979,0.202237,3558.0
ios,0.595930,0.111413,0.187729,1840.0
python,0.592841,0.101455,0.173259,2612.0
c++,0.438438,0.074338,0.127122,1964.0
c#,0.496109,0.073424,0.127916,3473.0
weighted avg,0.379913,0.062916,0.105741,33044.0
micro avg,0.471857,0.062916,0.111028,33044.0
samples avg,0.062856,0.058837,0.058605,33044.0
macro avg,0.289857,0.044122,0.074589,33044.0


## SVC

In [27]:
pipeline_svc= pipeline.Pipeline(
    [
        ("tfidf", pipeline_tfidf),
        (
            "OvRSVC",
            multiclass.OneVsRestClassifier(
                svm.SVC(cache_size=1080, max_iter=None if LONG else 10_000, random_state=42)
            ),
        ),
    ]
)

In [28]:
%%time
with ignore_warnings(category=exceptions.ConvergenceWarning):
    pipeline_svc.fit(
        X_train,
        y_train,
    )

CPU times: user 4h 1min 23s, sys: 1min 1s, total: 4h 2min 25s
Wall time: 3h 59min 19s


In [29]:
%%time
pred_svc = pipeline_svc.predict(X_test)
pred_svc

CPU times: user 1h 35min 29s, sys: 1.2 s, total: 1h 35min 30s
Wall time: 1h 35min 30s


array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

### Score et enregistrement

In [30]:
scores_svc = save_score(y_test,pred_svc,target_names,"TfidfOvRestSvc")
joblib.dump(pipeline_svc,"TfidfOvRestSvc_model.joblib")
scores_svc.sort_values(by="precision", ascending=False)

,precision,recall,f1-score,support
python,0.923077,0.004594,0.009143,2612.0
android,0.916667,0.021516,0.042045,2045.0
ios,0.733333,0.005978,0.011860,1840.0
weighted avg,0.207611,0.306712,0.070095,33044.0
macro avg,0.139371,0.140017,0.033035,33044.0
java,0.120142,0.967960,0.213754,3558.0
c#,0.119226,0.960265,0.212116,3473.0
micro avg,0.118569,0.306712,0.171023,33044.0
samples avg,0.116454,0.254149,0.153356,33044.0
javascript,0.114352,0.980036,0.204807,3356.0


# Word2Vec

In [31]:
%%time
print("Build & train Word2Vec model ...")
# sentences = [gensim.utils.simple_preprocess(text) for text in df[0:20_000]["Token"]]
# sentences = X_train
w2v_min_count = 1
w2v_size = 500
w2v_window = 5
w2v_epochs = 100
maxlen = 30
w2v_model = gensim.models.Word2Vec(
    min_count=w2v_min_count, window=w2v_window, vector_size=w2v_size, seed=42, workers=1
)
#                                                workers=multiprocessing.cpu_count())
w2v_model.build_vocab(X_train)
w2v_model.train(X_train, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)
model_vectors = w2v_model.wv
w2v_words = model_vectors.index_to_key
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
x_sentences_train = keras.preprocessing.sequence.pad_sequences(
    tokenizer.texts_to_sequences(X_train), maxlen=maxlen, padding="post"
)
x_sentences_test = keras.preprocessing.sequence.pad_sequences(
    tokenizer.texts_to_sequences(X_test), maxlen=maxlen, padding="post"
)
num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)

# matrice d'embedding
print("Create Embedding matrix ...")
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, w2v_size))
i = 0
j = 0

for word, idx in word_index.items():
    i += 1
    if word in w2v_words:
        j += 1
        embedding_vector = model_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[idx] = model_vectors[word]

word_rate = np.round(j / i, 4)
print("Word embedding rate : ", word_rate)
print("Embedding matrix: %s" % str(embedding_matrix.shape))

Build & train Word2Vec model ...
Vocabulary size: 739
Word2Vec trained
Number of unique words: 163658
Create Embedding matrix ...
Word embedding rate :  0.0018
Embedding matrix: (163658, 500)
CPU times: user 22min 47s, sys: 15.2 s, total: 23min 2s
Wall time: 22min 53s


## Creation du modèle

In [32]:
%%time
input = keras.layers.Input(shape=(len(x_sentences_train), maxlen), dtype="float64")
word_input = keras.layers.Input(shape=(maxlen,), dtype="float64")
word_embedding = keras.layers.Embedding(
    input_dim=vocab_size,
    output_dim=w2v_size,
    weights=[embedding_matrix],
    input_length=maxlen,
)(word_input)
word_vec = keras.layers.GlobalAveragePooling1D()(word_embedding)
embed_model = keras.models.Model([word_input], word_vec)

embed_model.summary()

2022-12-23 02:31:14.212199: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 500)           81829000  
_________________________________________________________________
global_average_pooling1d (Gl (None, 500)               0         
Total params: 81,829,000
Trainable params: 81,829,000
Non-trainable params: 0
_________________________________________________________________
CPU times: user 921 ms, sys: 690 ms, total: 1.61 s
Wall time: 678 ms


In [33]:
%%time
embeddings = embed_model.predict(x_sentences_train)
embeddings.shape

2022-12-23 02:31:15.023976: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


CPU times: user 3.57 s, sys: 472 ms, total: 4.05 s
Wall time: 2.86 s


(88831, 500)

In [34]:
%%time
embeddings_test = embed_model.predict(x_sentences_test)
embeddings_test.shape

CPU times: user 1.11 s, sys: 132 ms, total: 1.25 s
Wall time: 789 ms


(29611, 500)

In [35]:
%%time
with ignore_warnings(category=exceptions.ConvergenceWarning):
    OvsRSVC_word2vec = multiclass.OneVsRestClassifier(
        svm.SVC(cache_size=1080, max_iter=10_000 if LONG else 100, random_state=42)
    ).fit(embeddings, y_train)
    #    ).fit(embeddings, tags_are_in(df[0:10_000], target_names))

CPU times: user 1min 37s, sys: 2.03 s, total: 1min 39s
Wall time: 1min 39s


In [36]:
%%time
pred_word2vec = OvsRSVC_word2vec.predict(embeddings_test)
pred_word2vec

CPU times: user 38.9 s, sys: 5.03 ms, total: 38.9 s
Wall time: 38.9 s


array([[1, 0, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 1, 1]])

In [37]:
# scores_word2vec = save_score(
#     tags_are_in(df[0:10_000], target_names), pred_word2vec, target_names, "word2vec"
# )
scores_word2vec = save_score(y_test,pred_word2vec,target_names, "word2vec")
scores_word2vec.sort_values(by="precision", ascending=False)

,precision,recall,f1-score,support
c#,0.338184,0.194069,0.246615,3473.0
java,0.120329,0.893198,0.212086,3558.0
javascript,0.116071,0.928188,0.206339,3356.0
weighted avg,0.096959,0.773211,0.131413,33044.0
python,0.085328,0.877489,0.155532,2612.0
android,0.079929,0.022005,0.034509,2045.0
macro avg,0.064504,0.826979,0.096451,33044.0
ios,0.063977,0.916848,0.119607,1840.0
c++,0.063242,0.859470,0.117815,1964.0
html,0.061781,0.927066,0.115842,1755.0


# BERT

# USE